In [7]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=_31dZTlFh2JRwY1LDIJ1QlZFzJ6XVZQqhmW1VPllTj0&tc=RDv6wjd-rtjC0yU4hOkq2oKGVukmIMvucEzwFxL9xoo&cc=At_cL69N3NXYPluXwUuqROUqSJZvdd0XAxYRuHjJvXk

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VMFzyryqj8O_j9fXNwiI-X9ysOYD6FcgqercwZcs7EbGeaXf-7rS44


KeyError: ignored

# AOI

In [ ]:
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "MultiPolygon",
        "coordinates": [
          [
            [
              51.989783,
              35.881821
            ],
            [
              51.997926,
              35.901256
            ],
            [
              51.978882,
              35.904028
            ],
            [
              51.973603,
              35.883281
            ],
            [
              51.989783,
              35.881821
            ]
          ]
        ]
      }
    }
  ]
}

coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

# Visualization function

In [3]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

def visualize_image(col):
    # Mosaic the image collection.
    img = col.mosaic()

    # Create a folium map object.
    center = aoi.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=12, height=500, width=500)

    # Add layers to the folium map.
    m.add_ee_layer(img,
                   {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                   'S2 image', True, 1, 9)

    # Add a layer control panel to the map.
    m.add_child(folium.LayerControl())

    # Display the map.
    display(m)

# 1.1 Download Function

In [5]:
import matplotlib.pyplot as plt
import IPython.display as disp
%matplotlib inline

def download_SAR(aoi, start_date, end_date, number_of_image='all', visualize=True):
  """
  start and end date format: 'YYYY-MM-DD'
  number_of_image: 'all' or 'first'
  """
  import requests


  collection = ee.ImageCollection('COPERNICUS/S1_GRD')\
                  .filterBounds(aoi)\
                  .filterDate(ee.Date(start_date), ee.Date(end_date))\
                  .filterMetadata('transmitterReceiverPolarisation', 'equals', ['VV', 'VH'])

  sen2col = ee.ImageCollection('COPERNICUS/S2')\
                  .filterBounds(aoi)\
                  .filterDate(ee.Date(start_date), ee.Date(end_date))\
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))



  if number_of_image=='first':
    img = ee.Image(collection.first()).clip(aoi)
    link = img.getDownloadURL({
      'scale':30,
      'crs':'EPSG:4326',
      'fileFormat': 'GeoTIFF',
      'region':aoi
      })
    dl = requests.get(link, allow_redirects=True)
    filename = img.getInfo()['id'].split('_')[5][:8]
    open(filename, 'wb').write(dl.content)

    if visualize:
      print(f"Image on Date: {filename}")
      visualize_image(sen2col)


  else:
    count = collection.size().getInfo()
    for i in range(count):
      img = ee.Image(collection.toList(1,i).get(0)).clip(aoi)
      new_img = ee.Image.rgb(img.select('VV'), img.select('VH'), img.select('VV').divide(img.select('VH')))
      link = new_img.getDownloadURL({
          'scale':30,
          'crs':'EPSG:4326',
          'fileFormat': 'GeoTIFF',
          'filePerBand':False,
          'region':aoi
          })
      dl = requests.get(link, allow_redirects=True)
      filename = img.getInfo()['id'].split('_')[5][:8]
      open(f"{filename}.zip", 'wb').write(dl.content)

      if visualize:
        print(f"Image on Date: {filename}")
        visualize_image(sen2col)

# 1.2 Use download_sen1() function and visualize sentinel-2 image on that location

date format: 'YYYY-MM-DD'

images will be save on the left directory (wait for a while then you can download those files

In [6]:
download_SAR(aoi, "2019-08-01", "2019-08-30", number_of_image='all', visualize=False)

KeyboardInterrupt: ignored